In [8]:
import numpy as np
import pandas as pd

# Define constants for the game
num_moves = 3  # Number of possible moves: Rock (0), Paper (1), Scissors (2)
num_outcomes = 3  # Number of possible outcomes: Win (0), Lose (1), Tie (2)
state_short = ['r', 'p', 's']  # Short form representation for moves (Rock, Paper, Scissors)
states = ['Rock', 'Paper', 'Scissors']  # Full names for moves
outcomes = ['Win', 'Lose', 'Tie']  # Possible outcomes

# Load training data from a file (format: previous_move, current_move, outcome)
data_file = 'training.txt'  # Path to the training data file (update this as necessary)
data = np.loadtxt(data_file)  # Load data from the file into a NumPy array

# Initialize matrices for tracking move transitions and outcomes
transition_counts = np.zeros((num_moves, num_moves))  # Transition: prev_move → curr_move (counts)
observation_counts = np.zeros((num_outcomes, num_moves))  # Observation: outcome → curr_move (counts)

# Outcome mapping: Adjusting for the format you provided
# Original outcome format: 1 = Player wins, 0 = Tie, -1 = Player loses
outcome_map = {-1: 1, 0: 2, 1: 0}  # Map -1 -> Lose, 0 -> Tie, 1 -> Win

# Populate the matrices based on the training data
for entry in data:
    prev_move = int(entry[0])  # Previous move (0: Rock, 1: Paper, 2: Scissors)
    curr_move = int(entry[1])  # Current move
    outcome = int(entry[2])  # Outcome (1: Win, 0: Tie, -1: Lose)

    # Map outcome to [0: Win, 1: Lose, 2: Tie]
    if outcome in outcome_map:
        mapped_outcome = outcome_map[outcome]  # Map outcome to [0: Win, 1: Lose, 2: Tie]
    else:
        continue  # Skip invalid outcomes if any

    # Update the transition matrix (prev_move → curr_move)
    transition_counts[prev_move, curr_move] += 1
    # Update the observation matrix (mapped_outcome → curr_move)
    observation_counts[mapped_outcome, curr_move] += 1

# Convert frequency counts into probability matrices (rows should sum to 1)
transition_matrix = transition_counts / transition_counts.sum(axis=1, keepdims=True)
observation_matrix = observation_counts / observation_counts.sum(axis=1, keepdims=True)

# Function for dynamically updating the matrices based on new data
def update_matrices(prev_move, curr_move, outcome):
    """Update transition and observation matrices based on a new move and outcome."""
    global transition_counts, observation_counts, transition_matrix, observation_matrix

    # Map the outcome (1 = Win, 0 = Tie, -1 = Lose) to the new format (0 = Win, 1 = Lose, 2 = Tie)
    if outcome in outcome_map:
        mapped_outcome = outcome_map[outcome]
    else:
        return  # If the outcome is invalid, do nothing

    # Update the frequency counts in both matrices
    transition_counts[prev_move, curr_move] += 1
    observation_counts[mapped_outcome, curr_move] += 1

    # Normalize the matrices again to get updated probabilities
    transition_matrix = transition_counts / transition_counts.sum(axis=1, keepdims=True)
    observation_matrix = observation_counts / observation_counts.sum(axis=1, keepdims=True)

# Implement Viterbi algorithm to predict the sequence of moves
def viterbi_algorithm(observed_sequence):
    """Run the Viterbi algorithm to predict the most likely sequence of moves."""
    # Initialize the Viterbi matrix and backpointer matrix
    V = np.zeros((len(observed_sequence), num_moves))  # Viterbi matrix (storing probabilities)
    backpointer = np.zeros((len(observed_sequence), num_moves), dtype=int)  # Backpointer for path reconstruction

    # Initialization step
    for state in range(num_moves):
        V[0, state] = observation_matrix[observed_sequence[0], state] * (1 / num_moves)  # Assuming uniform initial probabilities

    # Recursion step
    for t in range(1, len(observed_sequence)):
        for state in range(num_moves):
            # Compute the maximum probability for this state at time t
            max_prob = -1
            max_state = -1
            for prev_state in range(num_moves):
                prob = V[t-1, prev_state] * transition_matrix[prev_state, state] * observation_matrix[state, observed_sequence[t]]
                if prob > max_prob:
                    max_prob = prob
                    max_state = prev_state
            V[t, state] = max_prob
            backpointer[t, state] = max_state

    # Backtracking to find the most likely sequence of moves
    best_sequence = []
    best_last_state = np.argmax(V[-1, :])
    best_sequence.append(best_last_state)
    for t in range(len(observed_sequence) - 1, 0, -1):
        best_last_state = backpointer[t, best_last_state]
        best_sequence.append(best_last_state)
    best_sequence.reverse()

    return best_sequence

# Main game loop: Player plays against AI
def play_game():
    """Main loop where the human player plays against the AI."""
    print("Play the game: 'r' for Rock, 'p' for Paper, 's' for Scissors. Press 'q' to quit.")

    prev_move = None  # Previous move of the player (to help predict future moves)
    prev_outcome = None  # Outcome of the previous round (to help predict future moves)

    observed_sequence = []  # To track the sequence of outcomes

    while True:
        # Get the player's move (input validation)
        player_input = input("Your move: ").strip().lower()

        # Exit condition if player presses 'q'
        if player_input == 'q':
            break
        # Check if the player input is valid (either 'r', 'p', or 's')
        elif player_input not in state_short:
            print("Invalid input. Use 'r', 'p', or 's'.")
            continue

        # Convert player's move into an index (0: Rock, 1: Paper, 2: Scissors)
        curr_move = state_short.index(player_input)

        # Predict the next sequence of moves based on observed outcomes
        if observed_sequence:
            predicted_moves = viterbi_algorithm(observed_sequence)
            print("Predicted sequence of moves:", [states[move] for move in predicted_moves])
        else:
            predicted_moves = [np.random.choice(num_moves)]  # Random move if no history

        # AI's move: Choose the move that will beat the predicted player move
        computer_move = (predicted_moves[0] + 1) % 3  # AI plays the winning move

        # Determine the outcome of the round
        if computer_move == curr_move:
            outcome = 2  # Tie
        elif (computer_move - curr_move) % 3 == 1:
            outcome = 1  # AI wins
        else:
            outcome = 0  # Player wins

        # Display round results
        print(f"Round Result: You played {states[curr_move]}, Computer played {states[computer_move]}")
        print(f"Outcome: {outcomes[outcome]}\n")

        # Update the observed sequence with the outcome
        observed_sequence.append(outcome)

        # Update matrices dynamically after each round
        if prev_move is not None and prev_outcome is not None:
            update_matrices(prev_move, curr_move, prev_outcome)

        # Set previous move and outcome for use in the next round
        prev_move = curr_move
        prev_outcome = outcome

# Start the game
if __name__ == "__main__":
    play_game()  # Begin the game loop


Play the game: 'r' for Rock, 'p' for Paper, 's' for Scissors. Press 'q' to quit.
Round Result: You played Rock, Computer played Rock
Outcome: Tie

Predicted sequence of moves: ['Rock']
Round Result: You played Paper, Computer played Paper
Outcome: Tie

Predicted sequence of moves: ['Scissors', 'Paper']
Round Result: You played Scissors, Computer played Rock
Outcome: Lose

Predicted sequence of moves: ['Rock', 'Rock', 'Rock']
Round Result: You played Rock, Computer played Paper
Outcome: Lose

Predicted sequence of moves: ['Rock', 'Rock', 'Rock', 'Rock']
Round Result: You played Paper, Computer played Paper
Outcome: Tie

Predicted sequence of moves: ['Rock', 'Rock', 'Rock', 'Rock', 'Paper']
Round Result: You played Scissors, Computer played Paper
Outcome: Win

Predicted sequence of moves: ['Rock', 'Rock', 'Rock', 'Rock', 'Paper', 'Scissors']
Round Result: You played Rock, Computer played Paper
Outcome: Lose

Predicted sequence of moves: ['Rock', 'Rock', 'Rock', 'Rock', 'Paper', 'Scissors

In [32]:
########################################################################

# This is to claculate percision

import numpy as np

# Constants for the game
num_moves = 3  # Number of possible moves: Rock (0), Paper (1), Scissors (2)
state_short = ['r', 'p', 's']  # Short form representation for moves (Rock, Paper, Scissors)
states = ['Rock', 'Paper', 'Scissors']  # Full names for moves
outcomes = ['Win', 'Lose', 'Tie']  # Possible outcomes

# Initialize matrices for tracking move transitions and outcomes
transition_counts = np.zeros((num_moves, num_moves))  # Transition: prev_move → curr_move (counts)
observation_counts = np.zeros((num_outcomes, num_moves))  # Observation: outcome → curr_move (counts)

# Outcome mapping: Adjusting for the format you provided
outcome_map = {-1: 1, 0: 2, 1: 0}  # Map -1 -> Lose, 0 -> Tie, 1 -> Win

# Probability matrices (transition and observation)
transition_matrix = np.zeros((num_moves, num_moves))
observation_matrix = np.zeros((num_outcomes, num_moves))

# Function for dynamically updating the matrices based on new data
def update_matrices(prev_move, curr_move, outcome):
    """Update transition and observation matrices based on a new move and outcome."""
    global transition_counts, observation_counts, transition_matrix, observation_matrix

    # Map the outcome (1 = Win, 0 = Tie, -1 = Lose) to the new format (0 = Win, 1 = Lose, 2 = Tie)
    if outcome in outcome_map:
        mapped_outcome = outcome_map[outcome]
    else:
        return  # If the outcome is invalid, do nothing

    # Update the frequency counts in both matrices
    transition_counts[prev_move, curr_move] += 1
    observation_counts[mapped_outcome, curr_move] += 1

    # Avoid division by zero: Check if rows sum to 0, and prevent division by zero
    transition_row_sums = transition_counts.sum(axis=1, keepdims=True)
    observation_row_sums = observation_counts.sum(axis=1, keepdims=True)

    # Update the probability matrices (with checks for division by zero)
    transition_matrix = np.divide(transition_counts, transition_row_sums, where=transition_row_sums != 0)
    observation_matrix = np.divide(observation_counts, observation_row_sums, where=observation_row_sums != 0)

# Viterbi algorithm (same as before, but used to predict the sequence)
def viterbi_algorithm(observed_sequence):
    """Run the Viterbi algorithm to predict the most likely sequence of moves."""
    if len(observed_sequence) == 0:
        return []  # If no observations, return an empty prediction

    V = np.zeros((len(observed_sequence), num_moves))  # Viterbi matrix (storing probabilities)
    backpointer = np.zeros((len(observed_sequence), num_moves), dtype=int)  # Backpointer for path reconstruction

    # Initialization step
    for state in range(num_moves):
        V[0, state] = observation_matrix[observed_sequence[0], state] * (1 / num_moves)

    # Recursion step
    for t in range(1, len(observed_sequence)):
        for state in range(num_moves):
            max_prob = -1
            max_state = -1
            for prev_state in range(num_moves):
                prob = V[t-1, prev_state] * transition_matrix[prev_state, state] * observation_matrix[state, observed_sequence[t]]
                if prob > max_prob:
                    max_prob = prob
                    max_state = prev_state
            V[t, state] = max_prob
            backpointer[t, state] = max_state

    # Backtracking to find the most likely sequence of moves
    best_sequence = []
    best_last_state = np.argmax(V[-1, :])
    best_sequence.append(best_last_state)
    for t in range(len(observed_sequence) - 1, 0, -1):
        best_last_state = backpointer[t, best_last_state]
        best_sequence.append(best_last_state)
    best_sequence.reverse()

    return best_sequence

# Simulate 100 random games
def simulate_random_games(num_games=100):
    correct_predictions = 0
    total_games = num_games
    observed_sequence = []  # To track the sequence of outcomes

    for _ in range(num_games):
        # Random opponent move
        opponent_move = np.random.choice(num_moves)

        # Predict the next move using the Viterbi algorithm
        if observed_sequence:
            predicted_moves = viterbi_algorithm(observed_sequence)
            predicted_move = predicted_moves[0]  # Take the first prediction
        else:
            predicted_move = np.random.choice(num_moves)  # Random move if no history

        # AI's move: Choose the move that will beat the predicted player move
        computer_move = (predicted_move + 1) % 3  # AI plays the winning move

        # Randomly determine the outcome (Win, Lose, Tie)
        if computer_move == opponent_move:
            outcome = 2  # Tie
        elif (computer_move - opponent_move) % 3 == 1:
            outcome = 1  # AI wins
        else:
            outcome = 0  # Player wins

        # Update the observed sequence with the outcome
        observed_sequence.append(outcome)

        # Update matrices dynamically after each round
        update_matrices(opponent_move, computer_move, outcome)

        # Check if the AI's prediction was correct (AI predicted the opponent's move correctly)
        if computer_move == opponent_move:
            correct_predictions += 1

    accuracy = correct_predictions / total_games
    return accuracy

# Calculate precision (the ratio of correct predictions for ties, wins, and losses)
def calculate_precision():
    total_true_positives = 0
    total_predicted_positives = 0
    total_actual_positives = 0

    for _ in range(100):  # For simplicity, we are simulating 100 rounds
        opponent_move = np.random.choice(num_moves)
        predicted_moves = viterbi_algorithm([])  # Use the empty observed sequence for predictions
        predicted_move = predicted_moves[0] if predicted_moves else np.random.choice(num_moves)

        computer_move = (predicted_move + 1) % 3
        if computer_move == opponent_move:
            total_true_positives += 1  # AI predicted tie correctly
            total_predicted_positives += 1

        total_actual_positives += 1

    precision = total_true_positives / total_predicted_positives if total_predicted_positives else 0
    return precision

# Run the game simulation and compute accuracy and precision
accuracy = simulate_random_games(num_games=100)
precision = calculate_precision()

print(f"Model accuracy: {accuracy * 100:.2f}%")
print(f"Model precision: {precision * 100:.2f}%")


Model accuracy: 41.00%
Model precision: 100.00%
